In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
mwdd_info = np.loadtxt('gaia_mwdd.csv',delimiter=',',dtype=str)
sample_info = np.loadtxt('ps1matches.txt',delimiter=',',dtype=str)

In [3]:
mwdd_info[:,0]

array(['request', 'Gaia DR2 1038076351151065088',
       'Gaia DR2 1058284412796260480', ..., 'Gaia DR2 998347456987797120',
       'Gaia DR2 99915890086770176', 'Gaia DR2 999298152291419776'],
      dtype='<U28')

In [45]:
# this part is to make sure crossmatches are good using atmosphere models
#parallax.f info (PS1+gaia)
pid = []
chisq = []


with open('OUT_parallax_PanSTARRS_100pc_H') as f:
    lignes = f.readlines()
    for i in range(len(lignes)):
        ligne = lignes[i]
        
        if ligne[:8]=='chisq = ':
            chisq.append(ligne[8:-2].strip())
            
        elif ligne[:5]=='-----':
            nameline = lignes[i-1][:-2].rstrip()
            pid.append(nameline)
            
# tag matches according to their chisq
qflags = []

for i in range(len(chisq)):
    if chisq[i]=='Na': 
        qflags.append('bad')
        continue
    
    q = float(chisq[i])
    if q==0.: qflags.append('noPhot')
    elif q<0.0005: qflags.append('bad')
    elif q<=10.: qflags.append('good')
    elif q<=100.: qflags.append('suspect')
    else: qflags.append('bad')
        
print(qflags.count('good'),qflags.count('suspect'),qflags.count('bad'),qflags.count('noPhot'))
print(qflags.count('bad')/(len(qflags))*100)

4461 4103 746 58
7.96327924850555


In [46]:
# this part is to make sure crossmatches are good using atmosphere models
#parallax.f info (PS1+gaia)
pid_mwdd = []
chisq_mwdd = []


with open('OUT_parallax_PanSTARRS_100pc_H_mwdd') as f:
    lignes = f.readlines()
    for i in range(len(lignes)):
        ligne = lignes[i]
        
        if ligne[:8]=='chisq = ':
            chisq_mwdd.append(ligne[8:-2].strip())
            
        elif ligne[:-1]=='------------------------------':
            nameline = lignes[i-1][:-2].rstrip()
            pid_mwdd.append(nameline)
            
# tag matches according to their chisq
qflags_mwdd = []

for i in range(len(chisq_mwdd)):
    if chisq_mwdd[i]=='Na': 
        qflags_mwdd.append('bad')
        continue
    
    q = float(chisq_mwdd[i])
    if q==0.: qflags_mwdd.append('noPhot')
    elif q<0.0005: qflags_mwdd.append('bad')
    elif q<=10.: qflags_mwdd.append('good')
    elif q<=100.: qflags_mwdd.append('suspect')
    else: qflags_mwdd.append('bad')
        
print(qflags_mwdd.count('good'),qflags_mwdd.count('suspect'),qflags_mwdd.count('bad'),qflags_mwdd.count('noPhot'))
print(qflags_mwdd.count('bad')/(len(qflags_mwdd))*100)

7183 294 45 5278
0.3515625


In [47]:
ctr_NotInMWDD = 0
ctr_NoPS1InMWDD = 0
ctr_DiffPhot = 0
ctr_DiffBad = 0

for i in range(1,len(sample_info)):
    obj = sample_info[i]
    GaiaID = obj[0]
#    if i%100==0: print('({}/{})...'.format(i,len(sample_info)),end="")
    
    # find sample object in MWDD
    idx = np.flatnonzero(np.core.defchararray.find(mwdd_info[:,0],GaiaID)!=-1)
    
    # find sample object's quality flag
    qidx = pid.index(GaiaID[9:])
    qflag = qflags[qidx]
    
    if len(idx)<1: # if the object isn't in the MWDD, skip it
#        print('Object not in MWDD')
        ctr_NotInMWDD += 1
        continue
    
    # compare PS1 photometry
    SamplePhot = np.asarray(obj[1:11:2],dtype=float) # sample photometry
    
    # sometimes there are bad PS1 points. set their value to 0
    MWDDPhot = mwdd_info[idx][0][-5:]
    MWDDPhot[np.where(MWDDPhot=='')] = 0.
    MWDDPhot = np.asarray(mwdd_info[idx][0][-5:],dtype=float) # MWDD photometry
    
    # check if there is MWDD phot
    if np.mean(MWDDPhot)==0:
        ctr_NoPS1InMWDD += 1
        continue
        
    qidx = pid_mwdd.index(GaiaID[9:])
    qflag_mwdd = qflags_mwdd[qidx]
    
    # check the difference. tolerate a 0.1 mag max diff
    PhotDiff = np.abs(SamplePhot-MWDDPhot)
    if np.any(PhotDiff>0.1):
        ctr_DiffPhot += 1
        if qflag=='bad': 
            ctr_DiffBad += 1
        else: print(GaiaID, qflag, qflag_mwdd)
#            print('Different photometry')

Gaia DR2 431635455820288128 noPhot good
Gaia DR2 2418116963320446720 noPhot good
Gaia DR2 2346642484109864960 good good
Gaia DR2 2342218942669916672 suspect good
Gaia DR2 2552928187080872832 noPhot suspect
Gaia DR2 426122397136335872 noPhot good
Gaia DR2 2790494850219788160 noPhot good
Gaia DR2 413330854959153792 good bad
Gaia DR2 2482813425794003200 good good
Gaia DR2 2482813430089468160 good good
Gaia DR2 105240786245136256 noPhot good
Gaia DR2 5135133886317601280 suspect good
Gaia DR2 2490975272405858048 good good
Gaia DR2 561231932144411008 noPhot good
Gaia DR2 339492052075797376 noPhot good
Gaia DR2 19693180966870656 suspect bad
Gaia DR2 8578256576520320 noPhot good
Gaia DR2 5188044687948351872 suspect good
Gaia DR2 5181816233750415488 good good
Gaia DR2 544718680539607680 suspect bad
Gaia DR2 3249114137765329920 good good
Gaia DR2 3249479592234269056 suspect good
Gaia DR2 66837563803594880 suspect good
Gaia DR2 490062094962403200 suspect suspect
Gaia DR2 490169022466344448 suspec

In [48]:
print(ctr_NotInMWDD,ctr_NoPS1InMWDD,ctr_DiffPhot, ctr_DiffBad)

40 1872 137 42


In [70]:
ctr_NotInSamp = 0

for i in range(1,len(mwdd_info)):
    obj = mwdd_info[i]
    if i%1000==0: print('({}/{})...'.format(i,len(mwdd_info)),end="")
    GaiaID = ''
    
    # extract mwdd photometry
    # sometimes there are bad PS1 points. set their value to 0
    MWDDPhot = obj[-5:]
    MWDDPhot[np.where(MWDDPhot=='')] = 0.
    MWDDPhot = np.asarray(obj[-5:],dtype=float) # MWDD photometry
    
    # if no MWDD photometry, skip
    if np.mean(MWDDPhot)==0.: 
#        print('No PS1')
        continue
        
    # else, look for Gaia ID
    if obj[0][:4]=='Gaia': # check main ID first
        GaiaID = obj[0]
        
    else: # else check all names
        AltNames = obj[5].split(';')
        for AltName in AltNames:
            if AltName[:4]=='Gaia': 
                GaiaID = AltName
                break
                
    # if object has no Gaia ID, skip
    if GaiaID=='': continue
        
    # else, compare with sample
    idx = np.flatnonzero(np.core.defchararray.find(sample_info[:,0],GaiaID[9:])!=-1) # find idx in sample
    
    if len(idx)==0: # if obj is not in sample
#        print('Object in MWDD but not in sample')
#        print('{} {}'.format(GaiaID,MWDDPhot))
        ctr_NotInSamp +=1
        continue
    
    SamplePhot = np.asarray(sample_info[idx][0][-5:],dtype=float) # sample photometry
    
    # check the difference. tolerate a 0.01 mag max diff
#    PhotDiff = np.abs(SamplePhot-MWDDPhot)
#    if np.any(PhotDiff>0.01):
#        print('Different photometry')
        

(1000/16269)...(2000/16269)...(3000/16269)...(4000/16269)...(5000/16269)...(6000/16269)...(7000/16269)...(8000/16269)...(9000/16269)...(10000/16269)...(11000/16269)...(12000/16269)...(13000/16269)...(14000/16269)...(15000/16269)...(16000/16269)...

In [71]:
print(ctr_NotInMWDD,ctr_NoPS1InMWDD,ctr_DiffPhot,ctr_NotInSamp)

64 1869 317 2545
